In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
from sqlalchemy import inspect
import sys
import sqlalchemy.ext.declarative as sqld

In [2]:
sys.path.append('..')

In [3]:
from modules.transforms import *

In [4]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [5]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [7]:
pop=pd.read_sql_table('producto1_Covid_19', schema='producto1',con=cnx) 
pop=pop.groupby('Region')['Poblacion'].sum()
popTOT=pop.sum()

In [8]:
popTOT

19458310.0

In [9]:
#FALLECIDOS DEIS
s="DEIS"
n="DEIS_2020"
DEIS=pd.read_sql_table(n, con=cnx,schema=s)
DEIS=DEIS[DEIS['COVID'].notnull()]
master=DEIS.groupby('FECHA_DEF')['index'].count().reset_index()
master.columns=['Fecha','Fallecidos DEIS Diarios']
master=master.merge(DEIS[DEIS['COVID']=='Covid PCR+'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (PCR+)'}),on='Fecha')
master=master.merge(DEIS[DEIS['COVID']=='Covid Sospechoso'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (Covid Sospechoso)'}),on='Fecha')
master['Fallecidos DEIS (PCR+) Acumulado']=master['Fallecidos DEIS Diarios (PCR+)'].cumsum()
master['Fallecidos DEIS (Covid Sospechoso) Acumulado']=master['Fallecidos DEIS Diarios (Covid Sospechoso)'].cumsum()
master['Fallecidos DEIS Acumulado']=master['Fallecidos DEIS Diarios'].cumsum()
master['Fallecidos DEIS (PCR+) x Millón/Hab.']=master['Fallecidos DEIS (PCR+) Acumulado']/(popTOT/1000000)
master['Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.']=master['Fallecidos DEIS (Covid Sospechoso) Acumulado']/(popTOT/1000000)
master['Fallecidos DEIS x Millón/Hab.']=master['Fallecidos DEIS Acumulado']/(popTOT/1000000)

In [10]:
master

,Fecha,Fallecidos DEIS Diarios,Fallecidos DEIS Diarios (PCR+),Fallecidos DEIS Diarios (Covid Sospechoso),Fallecidos DEIS (PCR+) Acumulado,Fallecidos DEIS (Covid Sospechoso) Acumulado,Fallecidos DEIS Acumulado,Fallecidos DEIS (PCR+) x Millón/Hab.,Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.,Fallecidos DEIS x Millón/Hab.
0,2020-03-20,2,1,1,1,1,2,0.051392,0.051392,0.102784
1,2020-03-21,2,1,1,2,2,4,0.102784,0.102784,0.205568
2,2020-03-25,2,1,1,3,3,6,0.154176,0.154176,0.308352
3,2020-03-26,2,1,1,4,4,8,0.205568,0.205568,0.411135
4,2020-03-27,4,1,3,5,7,12,0.256960,0.359743,0.616703
...,...,...,...,...,...,...,...,...,...,...
145,2020-08-15,90,48,42,10645,4015,14660,547.067037,206.338577,753.405614
146,2020-08-16,83,45,38,10690,4053,14743,549.379674,208.291470,757.671144
147,2020-08-17,96,52,44,10742,4097,14839,552.052054,210.552715,762.604769
148,2020-08-18,74,39,35,10781,4132,14913,554.056339,212.351432,766.407771


In [11]:
#casos activos según fecha de inicio de síntomas
s="producto53"
n="producto53_confirmados_"
activos=pd.read_sql_table(n, con=cnx,schema=s)

In [12]:
activos.groupby('fecha')

In [13]:
activos

,index,fecha,region,servicio.salud,confirmados.1,confirmados.2,confirmados.3,confirmados.4,confirmados.5,confirmados.6,...,confirmados.191,confirmados.192,confirmados.193,confirmados.194,confirmados.195,confirmados.196,confirmados.197,confirmados.198,confirmados.199,confirmados.200
0,0,2020-03-01,15,151011,0,0,1,2,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,2020-03-02,15,151011,0,1,0,0,1,1,...,1,0,0,0,0,0,0,0,1,0
2,2,2020-03-03,15,151011,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1
3,3,2020-03-04,15,151011,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,2020-03-05,15,151011,0,0,0,2,0,0,...,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4828,4828,2020-08-22,9,81012,69,117,130,135,78,75,...,92,59,81,108,95,111,85,103,97,115
4829,4829,2020-08-23,9,81012,71,84,86,148,78,72,...,66,109,85,95,92,96,40,90,117,90
4830,4830,2020-08-24,9,81012,84,96,71,214,78,87,...,79,95,102,109,114,85,61,80,77,66
4831,4831,2020-08-25,9,81012,131,114,101,238,87,108,...,120,129,140,108,126,129,77,163,81,95
